In [0]:
MODELO = "NASNet-FineTune"

In [1]:
import os
import keras
keras.__version__
import numpy as np
import pandas as pd
import tensorflow as tf

from keras import optimizers
from keras.utils import multi_gpu_model
from tensorflow.python.client import device_lib

Using TensorFlow backend.


### Multi - GPU

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3"
multi_gpu = len("0,1,2,3".split(','))

In [3]:
def get_session():
    config = tf.ConfigProto(device_count = {'CPU' :24, 'GPU':4})
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

In [4]:
keras.backend.tensorflow_backend.set_session(get_session())

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

# Directorios

In [6]:
# El directorio de trabajo
processDir = './process'

# Directorio para entrenamiento, validacion y test
train_dir = os.path.join(processDir, 'train')
validation_dir = os.path.join(processDir, 'validation')
test_dir = os.path.join(processDir, 'test')


# Generador de datos

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(331, 331),
        batch_size=32,
        class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
        validation_dir,
        target_size=(331, 331),
        batch_size=32,
        class_mode='categorical')

Found 14530 images belonging to 120 classes.
Found 3025 images belonging to 120 classes.


# Modelo NASNet

In [8]:
from keras.applications.nasnet import NASNetLarge

if multi_gpu > 1:  
    with tf.device('/cpu:0'):
        conv_base = NASNetLarge(weights='imagenet',
                                include_top=False,
                                input_shape=(331, 331, 3))

343613440/343610240 [==============================] - 26s 0us/step


In [9]:
conv_base.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 331, 331, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_261 (Activation)     (None, 165, 165, 96) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

normal_add_4_9 (Add)            (None, 21, 21, 336)  0           normal_left4_9[0][0]             
                                                                 normal_right4_9[0][0]            
__________________________________________________________________________________________________
normal_add_5_9 (Add)            (None, 21, 21, 336)  0           separable_conv_2_bn_normal_left5_
                                                                 normal_bn_1_9[0][0]              
__________________________________________________________________________________________________
normal_concat_9 (Concatenate)   (None, 21, 21, 2016) 0           adjust_bn_9[0][0]                
                                                                 normal_add_1_9[0][0]             
                                                                 normal_add_2_9[0][0]             
                                                                 normal_add_3_9[0][0]             
          

__________________________________________________________________________________________________
activation_440 (Activation)     (None, 11, 11, 672)  0           separable_conv_1_bn_reduction_lef
__________________________________________________________________________________________________
activation_442 (Activation)     (None, 11, 11, 672)  0           separable_conv_1_bn_reduction_rig
__________________________________________________________________________________________________
separable_conv_2_reduction_left (None, 11, 11, 672)  468384      activation_440[0][0]             
__________________________________________________________________________________________________
separable_conv_2_reduction_righ (None, 11, 11, 672)  484512      activation_442[0][0]             
__________________________________________________________________________________________________
activation_443 (Activation)     (None, 21, 21, 672)  0           adjust_bn_reduce_12[0][0]        
__________

# Trainable Layers

In [10]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'normal_concat_17':
        set_trainable = True
    layer.trainable = set_trainable

In [11]:
# Después de modificar el atributo trainable
for layer in conv_base.layers:
  print(layer.name, layer.trainable)

input_2 False
stem_conv1 False
stem_bn1 False
activation_261 False
reduction_conv_1_stem_1 False
reduction_bn_1_stem_1 False
activation_262 False
activation_264 False
separable_conv_1_pad_reduction_left1_stem_1 False
separable_conv_1_pad_reduction_right1_stem_1 False
separable_conv_1_reduction_left1_stem_1 False
separable_conv_1_reduction_right1_stem_1 False
separable_conv_1_bn_reduction_left1_stem_1 False
separable_conv_1_bn_reduction_right1_stem_1 False
activation_263 False
activation_265 False
separable_conv_2_reduction_left1_stem_1 False
separable_conv_2_reduction_right1_stem_1 False
activation_266 False
separable_conv_2_bn_reduction_left1_stem_1 False
separable_conv_2_bn_reduction_right1_stem_1 False
separable_conv_1_pad_reduction_right2_stem_1 False
activation_268 False
reduction_add_1_stem_1 False
separable_conv_1_reduction_right2_stem_1 False
separable_conv_1_pad_reduction_right3_stem_1 False
activation_270 False
separable_conv_1_bn_reduction_right2_stem_1 False
separable_conv_

activation_490 False
activation_492 False
activation_494 False
separable_conv_1_normal_left1_16 False
separable_conv_1_normal_right1_16 False
separable_conv_1_normal_left2_16 False
separable_conv_1_normal_right2_16 False
separable_conv_1_normal_left5_16 False
separable_conv_1_bn_normal_left1_16 False
separable_conv_1_bn_normal_right1_16 False
separable_conv_1_bn_normal_left2_16 False
separable_conv_1_bn_normal_right2_16 False
separable_conv_1_bn_normal_left5_16 False
activation_487 False
activation_489 False
activation_491 False
activation_493 False
activation_495 False
separable_conv_2_normal_left1_16 False
separable_conv_2_normal_right1_16 False
separable_conv_2_normal_left2_16 False
separable_conv_2_normal_right2_16 False
separable_conv_2_normal_left5_16 False
separable_conv_2_bn_normal_left1_16 False
separable_conv_2_bn_normal_right1_16 False
separable_conv_2_bn_normal_left2_16 False
separable_conv_2_bn_normal_right2_16 False
normal_left3_16 False
normal_left4_16 False
normal_right

# Model

In [12]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(120, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
flatten_1 (Flatten)          (None, 487872)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               249790976 
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)              

# Optimizador

In [13]:
from keras_radam import RAdam

model.compile(loss='categorical_crossentropy',
                       optimizer=RAdam(total_steps=5000, warmup_proportion=0.1, min_lr=1e-7),
                       metrics=['categorical_accuracy'])

# Callbacks

In [14]:
from keras import callbacks

callbacks = [callbacks.EarlyStopping(monitor='val_categorical_accuracy', min_delta=0.001, patience=20, restore_best_weights=True, mode='max')]

# Entrenamiento

In [15]:
history = model.fit_generator(train_generator,
                                       steps_per_epoch=100,
                                       epochs=2000,
                                       validation_data=valid_generator,
                                       validation_steps=50,
                                       callbacks=callbacks)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Kike\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-9c8d7b5a0957>", line 6, in <module>
    callbacks=callbacks)
  File "C:\Users\Kike\Anaconda3\lib\site-packages\keras\legacy\interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\Kike\Anaconda3\lib\site-packages\keras\engine\training.py", line 1732, in fit_generator
    initial_epoch=initial_epoch)
  File "C:\Users\Kike\Anaconda3\lib\site-packages\keras\engine\training_generator.py", line 42, in fit_generator
    model._make_train_function()
  File "C:\Users\Kike\Anaconda3\lib\site-packages\keras\engine\training.py", line 333, in _make_train_function
    **self._function_kwargs)
  File "C:\Users\Kike\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py", line 3009, in function
    **kwargs)
  File "C:\Users\Kike\Anaconda3\lib

KeyboardInterrupt: 

In [0]:
import pickle

# open a file, where you ant to store the data
file = open('./models/history_{}.pkl'.format(MODELO), 'wb')

# dump information to that file
pickle.dump(history.history, file)

# close the file
file.close()

# Graficos

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Train')
plt.plot(epochs, val_acc, 'b', label='Validation')
plt.title('Accuracy: Train, Validation')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Train')
plt.plot(epochs, val_loss, 'b', label='Validation')
plt.title('Loss: Train, Validation')
plt.legend()

plt.show()

In [0]:
parallel_model.save('./models/model_{}_RAdam.h5'.format(MODELO))

# Predict

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode=None,
        shuffle=False)

In [0]:
probabilities = parallel_model.predict_generator(test_generator)

In [0]:
# Get Max Index
predicted_class_index=np.argmax(probabilities,axis=1)

# Dictionary: Index to Class
class_labels = list(test_generator.class_indices.keys())
index_to_labels = dict((v,k) for k,v in test_generator.class_indices.items())

# Get List of Predictions
predictions = [index_to_labels[ix] for ix in predicted_class_index]

# Get Dictionary of Predictions
results=pd.DataFrame({"Filename":test_generator.filenames,
                      "Predictions":predictions})

In [0]:
results.head()

In [0]:
results.to_csv("./models/results_details_{}.csv".format(MODELO), sep="|", index=False)

In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_generator.labels, predicted_class_index)

In [0]:
results_dict = {}
tp_sum, fp_sum = 0, 0

for ix, row in enumerate(cm):
    
    tp = row[ix]
    fp = np.sum(row) - tp   
    results_dict[ix] = [class_labels[ix],tp,fp,(tp/(tp+fp)*100)]

    tp_sum += tp
    fp_sum += fp
    
df_res = pd.DataFrame.from_dict(results_dict, orient="index", columns=["Class","True Positive","False Positive","Accuracy"])

print("General Accuracy:{:0.2f}".format(tp_sum/(tp_sum+fp_sum)))

In [0]:
df_res.head()

In [0]:
df_res.to_csv("./models/results_{}.csv".format(MODELO), sep="|", index=False)